In [1]:
# import libraries
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, confusion_matrix,\
recall_score, precision_score, accuracy_score, log_loss,\
roc_curve, roc_auc_score, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.model_selection import train_test_split, KFold, GridSearchCV,\
cross_val_score, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix

In [2]:
df = pd.read_csv('holdout_data.csv' , index_col=0)

In [3]:
df.rename(columns = {'X1':'credit_limit', 'X2':'gender', 'X3':'education', 'X4':'marital_status', 'X5':'age', 'X6':'repay_sep',\
                    'X7':'repay_aug', 'X8':'repay_jul', 'X9':'repay_jun', 'X10':'repay_may', 'X11':'repay_apr', 'X12':'bill_sep', 'X13':'bill_aug',\
                    'X14':'bill_jul', 'X15':'bill_jun', 'X16':'bill_may', 'X17':'bill_apr', 'X18':'pay_sep', 'X19':'pay_aug', 'X20':'pay_jul',\
                    'X21':'pay_jun', 'X22':'pay_may', 'X23':'pay_apr','Y':'default'}, inplace = True)

In [4]:
df.head()

,credit_limit,gender,education,marital_status,age,repay_sep,repay_aug,repay_jul,repay_jun,repay_may,...,bill_jul,bill_jun,bill_may,bill_apr,pay_sep,pay_aug,pay_jul,pay_jun,pay_may,pay_apr
5501,180000,2,2,1,44,0,0,0,0,0,...,170788,174764,162667,166953,10000,8000,7000,6000,7000,10000
28857,130000,2,2,1,48,-2,-2,-2,-2,-2,...,1487,1279,749,440,1240,1487,1279,749,440,849
11272,60000,2,1,1,43,-1,3,2,0,0,...,495,330,165,340,0,330,0,0,340,0
8206,240000,1,1,1,42,0,0,0,0,0,...,91027,51508,51127,0,20000,2213,1030,1023,6790,10893
6362,100000,2,2,1,28,2,0,0,0,0,...,70844,63924,57326,59654,3500,3003,1910,2400,3300,0


In [5]:
taiwan = df

In [6]:
taiwan['education'] = np.where((taiwan.education == 5) | (taiwan.education == 6) | (taiwan.education == 0), 4, taiwan.education)
taiwan['education'].value_counts()

2    3514
1    2666
3    1204
4     117
Name: education, dtype: int64

In [7]:
taiwan['marital_status'] = np.where((taiwan.marital_status == 0), 3, taiwan.marital_status)
taiwan['marital_status'].value_counts()

2    3938
1    3464
3      99
Name: marital_status, dtype: int64

In [8]:
taiwan['repay_sep'] = np.where((taiwan.repay_sep == -2) | (taiwan.repay_sep == 0), -1, taiwan.repay_sep)
taiwan['repay_aug'] = np.where((taiwan.repay_aug == -2) | (taiwan.repay_aug == 0), -1, taiwan.repay_aug)
taiwan['repay_jul'] = np.where((taiwan.repay_jul == -2) | (taiwan.repay_jul == 0), -1, taiwan.repay_jul)
taiwan['repay_jun'] = np.where((taiwan.repay_jun == -2) | (taiwan.repay_jun == 0), -1, taiwan.repay_jun)
taiwan['repay_may'] = np.where((taiwan.repay_may == -2) | (taiwan.repay_may == 0), -1, taiwan.repay_may)
taiwan['repay_apr'] = np.where((taiwan.repay_apr == -2) | (taiwan.repay_apr == 0), -1, taiwan.repay_apr)

In [9]:
taiwan['credit_limit'] = np.where((taiwan.credit_limit > 240_000), 350_000, taiwan.credit_limit)

In [10]:
# Married individuals with advanced degrees
taiwan['married_degree'] = np.where((taiwan.marital_status == 1) & ((taiwan.education == 1) | (taiwan.education == 2)), 1, 0)

# Single individuals with advanced degrees
taiwan['single_degree'] = np.where((taiwan.marital_status >= 2) & ((taiwan.education == 1) | (taiwan.education == 2)), 1, 0)

# Married individuals with HS education
taiwan['single_hs'] = np.where((taiwan.marital_status == 1) & (taiwan.education == 3), 1, 0)

# Single individuals with HS education
taiwan['married_hs'] = np.where((taiwan.marital_status >= 2) & (taiwan.education == 3), 1, 0)

# Married Men with advanced degrees 
taiwan['married_men_adv'] = np.where((taiwan.marital_status == 1) & (taiwan.education <= 2) & (taiwan.gender == 1), 1, 0)

# Single Men with advanced degrees 
taiwan['single_men_adv'] = np.where((taiwan.marital_status >= 2) & (taiwan.education <= 2) & (taiwan.gender == 1), 1, 0)

# Married Women with advanced degrees
taiwan['married_women_adv'] = np.where((taiwan.marital_status == 1) & (taiwan.education <= 2) & (taiwan.gender == 2), 1, 0)

# Single Women with advanced degrees
taiwan['single_women_adv'] = np.where((taiwan.marital_status >= 2) & (taiwan.education <= 2) & (taiwan.gender == 2), 1, 0)

# Married Men without HS diploma
taiwan['married_men_dropout'] = np.where((taiwan.marital_status == 1) & (taiwan.education == 4) & (taiwan.gender == 1), 1, 0)

# Single Men without HS diploma
taiwan['single_men_dropout'] = np.where((taiwan.marital_status >= 2) & (taiwan.education == 4) & (taiwan.gender == 1), 1, 0)

# Married Women without HS diploma
taiwan['married_women_dropout'] = np.where((taiwan.marital_status == 1) & (taiwan.education == 4) & (taiwan.gender == 2), 1, 0)

# Single Women without HS diploma
taiwan['single_women_dropout'] = np.where((taiwan.marital_status >= 2) & (taiwan.education == 4) & (taiwan.gender == 2), 1, 0)

# All individuals with delinquent status over 3 months where last bill was over the mean of 50,000NT
taiwan['delinquent'] = np.where((taiwan.repay_sep == 3) & (taiwan.bill_sep >= 50_000), 1, 0)

# All individuals perfect payments
taiwan['no_risk'] = np.where((taiwan.repay_sep == -1) & (taiwan.repay_aug == -1) & (taiwan.repay_jul == -1) & (taiwan.repay_jun == -1) & (taiwan.repay_may == -1) & (taiwan.repay_apr == -1), 1, 0)



### Pickle in Data from Test Notebook

In [11]:
import pickle

infile = open("model.pickle",'rb')
model = pickle.load(infile)
infile.close()

In [12]:
infile = open("other_info.pickle",'rb')
other_info = pickle.load(infile)
infile.close()

In [13]:
taiwan.head()

,credit_limit,gender,education,marital_status,age,repay_sep,repay_aug,repay_jul,repay_jun,repay_may,...,married_men_adv,single_men_adv,married_women_adv,single_women_adv,married_men_dropout,single_men_dropout,married_women_dropout,single_women_dropout,delinquent,no_risk
5501,180000,2,2,1,44,-1,-1,-1,-1,-1,...,0,0,1,0,0,0,0,0,0,1
28857,130000,2,2,1,48,-1,-1,-1,-1,-1,...,0,0,1,0,0,0,0,0,0,1
11272,60000,2,1,1,43,-1,3,2,-1,-1,...,0,0,1,0,0,0,0,0,0,0
8206,240000,1,1,1,42,-1,-1,-1,-1,-1,...,1,0,0,0,0,0,0,0,0,1
6362,100000,2,2,1,28,2,-1,-1,-1,-1,...,0,0,1,0,0,0,0,0,0,0


In [14]:
taiwan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7501 entries, 5501 to 10111
Data columns (total 37 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   credit_limit           7501 non-null   int64
 1   gender                 7501 non-null   int64
 2   education              7501 non-null   int64
 3   marital_status         7501 non-null   int64
 4   age                    7501 non-null   int64
 5   repay_sep              7501 non-null   int64
 6   repay_aug              7501 non-null   int64
 7   repay_jul              7501 non-null   int64
 8   repay_jun              7501 non-null   int64
 9   repay_may              7501 non-null   int64
 10  repay_apr              7501 non-null   int64
 11  bill_sep               7501 non-null   int64
 12  bill_aug               7501 non-null   int64
 13  bill_jul               7501 non-null   int64
 14  bill_jun               7501 non-null   int64
 15  bill_may               7501 non-nu

#### Isolating Object Data Types for OHE

In [16]:
objects = other_info[0]

In [21]:
taiwan[objects] = taiwan[objects].astype(str)

In [22]:
X_obj = taiwan.select_dtypes('object')

ohe = OneHotEncoder(
    drop='first',
    sparse=False)

dums = ohe.fit_transform(X_obj)
dums_df = pd.DataFrame(dums,
                       columns=ohe.get_feature_names(),
                       index=X_obj.index)

In [23]:
taiwan.shape

(7501, 37)

In [24]:
dums_df.describe()

,x0_2,x1_2,x1_3,x1_4,x2_2,x2_3,x3_1,x3_2,x3_3,x3_4,...,x13_1,x14_1,x15_1,x16_1,x17_1,x18_1,x19_1,x20_1,x21_1,x22_1
count,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,...,7501.000000,7501.000000,7501.00000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000,7501.000000
mean,0.605253,0.468471,0.160512,0.015598,0.524997,0.013198,0.125050,0.084655,0.011065,0.003333,...,0.135849,0.189175,0.22397,0.274897,0.002000,0.003333,0.005066,0.005199,0.003600,0.663778
std,0.488829,0.499038,0.367105,0.123922,0.499408,0.114131,0.330798,0.278387,0.104615,0.057639,...,0.342651,0.391674,0.41693,0.446492,0.044677,0.057639,0.071000,0.071923,0.059892,0.472447
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [25]:
dums_df.columns

Index(['x0_2', 'x1_2', 'x1_3', 'x1_4', 'x2_2', 'x2_3', 'x3_1', 'x3_2', 'x3_3',
       'x3_4', 'x3_5', 'x3_6', 'x3_7', 'x3_8', 'x4_1', 'x4_2', 'x4_3', 'x4_4',
       'x4_5', 'x4_6', 'x4_7', 'x5_2', 'x5_3', 'x5_4', 'x5_5', 'x5_6', 'x5_7',
       'x5_8', 'x6_2', 'x6_3', 'x6_4', 'x6_5', 'x6_7', 'x7_2', 'x7_3', 'x7_4',
       'x7_5', 'x7_6', 'x7_7', 'x8_2', 'x8_3', 'x8_4', 'x8_5', 'x8_6', 'x8_7',
       'x9_1', 'x10_1', 'x11_1', 'x12_1', 'x13_1', 'x14_1', 'x15_1', 'x16_1',
       'x17_1', 'x18_1', 'x19_1', 'x20_1', 'x21_1', 'x22_1'],
      dtype='object')

In [26]:
holdout_columns = ['x0_2', 'x1_2', 'x1_3', 'x1_4', 'x2_2', 'x2_3', 'x3_1', 'x3_2', 'x3_3',
       'x3_4', 'x3_5', 'x3_6', 'x3_7', 'x3_8', 'x4_1', 'x4_2', 'x4_3', 'x4_4',
       'x4_5', 'x4_6', 'x4_7', 'x5_2', 'x5_3', 'x5_4', 'x5_5', 'x5_6', 'x5_7',
       'x5_8', 'x6_2', 'x6_3', 'x6_4', 'x6_5', 'x6_7', 'x7_2', 'x7_3', 'x7_4',
       'x7_5', 'x7_6', 'x7_7', 'x8_2', 'x8_3', 'x8_4', 'x8_5', 'x8_6', 'x8_7',
       'x9_1', 'x10_1', 'x11_1', 'x12_1', 'x13_1', 'x14_1', 'x15_1', 'x16_1',
       'x17_1', 'x18_1', 'x19_1', 'x20_1', 'x21_1', 'x22_1']

In [27]:
train_columns = ['x0_2', 'x1_2', 'x1_3', 'x1_4', 'x2_2', 'x2_3', 'x3_1', 'x3_2', 'x3_3',
       'x3_4', 'x3_5', 'x3_6', 'x3_7', 'x3_8', 'x4_1', 'x4_2', 'x4_3', 'x4_4',
       'x4_5', 'x4_6', 'x4_7', 'x4_8', 'x5_1', 'x5_2', 'x5_3', 'x5_4', 'x5_5',
       'x5_6', 'x5_7', 'x5_8', 'x6_1', 'x6_2', 'x6_3', 'x6_4', 'x6_5', 'x6_6',
       'x6_7', 'x6_8', 'x7_2', 'x7_3', 'x7_4', 'x7_5', 'x7_6', 'x7_7', 'x7_8',
       'x8_2', 'x8_3', 'x8_4', 'x8_5', 'x8_6', 'x8_7', 'x8_8', 'x9_1', 'x10_1',
       'x11_1', 'x12_1', 'x13_1', 'x14_1', 'x15_1', 'x16_1', 'x17_1', 'x18_1',
       'x19_1', 'x20_1', 'x21_1', 'x22_1']

In [28]:
[x for x in train_columns if x not in holdout_columns]

['x4_8', 'x5_1', 'x6_1', 'x6_6', 'x6_8', 'x7_8', 'x8_8']

In [29]:
dums_df['x4_8'] = 0
dums_df['x5_1'] = 0
dums_df['x6_1'] = 0
dums_df['x6_6'] = 0
dums_df['x6_8'] = 0
dums_df['x7_8'] = 0
dums_df['x8_8'] = 0

In [30]:
new_objects = ['x4_8', 'x5_1', 'x6_1', 'x6_6', 'x6_8', 'x7_8', 'x8_8']
objects.extend(new_objects)

In [31]:
dums_df.shape

(7501, 66)

In [35]:
dums_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7501 entries, 5501 to 10111
Data columns (total 66 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x0_2    7501 non-null   float64
 1   x1_2    7501 non-null   float64
 2   x1_3    7501 non-null   float64
 3   x1_4    7501 non-null   float64
 4   x2_2    7501 non-null   float64
 5   x2_3    7501 non-null   float64
 6   x3_1    7501 non-null   float64
 7   x3_2    7501 non-null   float64
 8   x3_3    7501 non-null   float64
 9   x3_4    7501 non-null   float64
 10  x3_5    7501 non-null   float64
 11  x3_6    7501 non-null   float64
 12  x3_7    7501 non-null   float64
 13  x3_8    7501 non-null   float64
 14  x4_1    7501 non-null   float64
 15  x4_2    7501 non-null   float64
 16  x4_3    7501 non-null   float64
 17  x4_4    7501 non-null   float64
 18  x4_5    7501 non-null   float64
 19  x4_6    7501 non-null   float64
 20  x4_7    7501 non-null   float64
 21  x5_2    7501 non-null   float64
 

In [38]:
missing = ['x4_8', 'x5_1', 'x6_1', 'x6_6', 'x6_8', 'x7_8', 'x8_8']
dums_df[missing] = dums_df[missing].astype(str)

#### Isolating Numeric Data Types for Scaling

In [36]:
ss = other_info[1]

In [42]:
X_nums = taiwan.select_dtypes('int64')

# ss.fit(X_nums)
nums_df = pd.DataFrame(ss.transform(X_nums),
                      index=X_nums.index)
nums_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
5501,0.128010,0.925806,1.473509,1.635734,1.769913,2.034417,2.008624,2.141444,0.262938,0.083279,0.095911,0.076045,0.137637,0.265635
28857,-0.291856,1.358827,-0.690974,-0.669129,-0.651566,-0.650874,-0.649358,-0.644367,-0.263890,-0.183176,-0.219241,-0.254778,-0.285395,-0.245825
11272,-0.879670,0.817551,-0.684327,-0.681776,-0.665754,-0.665563,-0.658945,-0.646040,-0.338464,-0.230510,-0.289697,-0.301967,-0.291844,-0.293276
8206,0.631850,0.709296,0.280429,0.578991,0.629107,0.126597,0.177627,-0.651729,0.864341,-0.153474,-0.232957,-0.237516,0.124094,0.315546
6362,-0.543776,-0.806277,0.290285,0.352368,0.340433,0.318778,0.279388,0.346300,-0.127973,-0.121154,-0.184481,-0.150762,-0.100964,-0.293276


In [43]:
nums_df.shape

(7501, 14)

#### Concatinating Transformed Datasets

In [44]:
taiwan_clean = pd.concat([nums_df, dums_df], axis=1)
taiwan_clean.head()

,0,1,2,3,4,5,6,7,8,9,...,x20_1,x21_1,x22_1,x4_8,x5_1,x6_1,x6_6,x6_8,x7_8,x8_8
5501,0.128010,0.925806,1.473509,1.635734,1.769913,2.034417,2.008624,2.141444,0.262938,0.083279,...,0.0,0.0,1.0,0,0,0,0,0,0,0
28857,-0.291856,1.358827,-0.690974,-0.669129,-0.651566,-0.650874,-0.649358,-0.644367,-0.263890,-0.183176,...,0.0,0.0,1.0,0,0,0,0,0,0,0
11272,-0.879670,0.817551,-0.684327,-0.681776,-0.665754,-0.665563,-0.658945,-0.646040,-0.338464,-0.230510,...,0.0,0.0,0.0,0,0,0,0,0,0,0
8206,0.631850,0.709296,0.280429,0.578991,0.629107,0.126597,0.177627,-0.651729,0.864341,-0.153474,...,0.0,0.0,1.0,0,0,0,0,0,0,0
6362,-0.543776,-0.806277,0.290285,0.352368,0.340433,0.318778,0.279388,0.346300,-0.127973,-0.121154,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [45]:
dums_df.shape

(7501, 66)

In [46]:
nums_df.shape

(7501, 14)

In [47]:
final_answers = model.predict(taiwan_clean)

In [48]:
final_results = pd.DataFrame(final_answers)

In [49]:
final_results.to_csv('taiwan_bakeoff_frazer.csv')

In [50]:
final_results.head() 

,0
0,0
1,0
2,0
3,0
4,1


In [53]:
final_results.describe()

,0
count,7501.000000
mean,0.130916
std,0.337331
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [54]:
final_results.mean()

0    0.130916
dtype: float64